<a href="https://colab.research.google.com/github/mateo020/LLM-Course-Advisor/blob/main/UofTCoursesLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
website = 'https://utm.calendar.utoronto.ca/course-search'
result = requests.get(website)
content = result.text

soup = BeautifulSoup(content, 'lxml')
# print(soup.prettify())

In [ ]:
print(soup.prettify())

In [ ]:
courses = soup.find_all('div', class_='views-row')
course_data = []
# Loop through each course block and extract the relevant data
for course in courses:
    # Extract course title
    title_div = course.find('h3', class_='js-views-accordion-group-header')
    if title_div:
        title = title_div.get_text(strip=True)
    else:
        title = "N/A"

    # Extract course description
    description_div = course.find('div', class_='views-field-field-desc')
    if description_div:
        description = description_div.get_text(strip=True)
    else:
        description = "N/A"

    # Extract course prerequisites
    prereq_span = course.find('span', class_='views-field-field-prerequisite')
    if prereq_span:
        prerequisites = prereq_span.get_text(strip=True)
    else:
        prerequisites = "None"

    # Store the extracted information in a dictionary
    course_info = {
        'course': title,
        'Description': description,
        'Prerequisites': prerequisites
    }

    # Append the course information to the list
    course_data.append(course_info)

In [ ]:
course_data[0]

{'course': 'ANT101H5 • Introduction to Biological Anthropology and Archaeology',
 'Description': 'Anthropology is the global and holistic study of human biology and behaviour, and includes four subfields: biological anthropology, archaeology, sociocultural anthropology and linguistic anthropology. The material covered is directed to answering the question: What makes us human? This course is a survey of biological anthropology and archaeology.',
 'Prerequisites': 'None'}

In [ ]:

# Start scraping from the first page
current_page = 0
course_data = []

while True:
    # Build the URL for the current page
    url = f"{website}?page={current_page}"

    # Send a request to fetch the page content
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {current_page}. Status code: {response.status_code}")
        break

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'lxml')


    courses = soup.find_all('div', class_='views-row')

    # Loop through each course block and extract the relevant data
    for course in courses:
        # Extract course title
        title_div = course.find('h3', class_='js-views-accordion-group-header')
        if title_div:
            title = title_div.get_text(strip=True)
        else:
            title = "N/A"

        # Extract course description
        description_div = course.find('div', class_='views-field-field-desc')
        if description_div:
            description = description_div.get_text(strip=True)
        else:
            description = "N/A"

        # Extract course prerequisites
        prereq_span = course.find('span', class_='views-field-field-prerequisite')
        if prereq_span:
            prerequisites = prereq_span.get_text(strip=True)
        else:
            prerequisites = "None"

        # Store the extracted information in a dictionary
        course_info = {
            'Course': title,
            'Description': description,
            'Prerequisites': prerequisites
        }

        # Append the course information to the list
        course_data.append(course_info)

    # Find the 'next' page link
    next_page_link = soup.find('a', title="Go to next page")

    # If no next page is found, break the loop
    if not next_page_link:
        print("No more pages to scrape.")
        break


    current_page += 1



No more pages to scrape.


In [ ]:
len(course_data)

4878

In [ ]:
course_data[0]

{'Course': 'ANT101H5 • Introduction to Biological Anthropology and Archaeology',
 'Description': 'Anthropology is the global and holistic study of human biology and behaviour, and includes four subfields: biological anthropology, archaeology, sociocultural anthropology and linguistic anthropology. The material covered is directed to answering the question: What makes us human? This course is a survey of biological anthropology and archaeology.',
 'Prerequisites': 'None'}

Scraping program information

In [ ]:
website = 'https://utm.calendar.utoronto.ca/list-program-areas'
result = requests.get(website)
content = result.text

soup = BeautifulSoup(content, 'lxml')
print(soup.prettify())

In [ ]:
links = soup.find_all('td')
links

In [ ]:
program_list = []
for td in links:
    a_tag = td.find('a')
    if a_tag and 'href' in a_tag.attrs:
        href = a_tag['href']
        # Construct the full URL (if it's a relative URL)
        full_url = requests.compat.urljoin(url, href)
        program_list.append(full_url)

In [ ]:
len(program_list)

81

In [ ]:
program_list[0]

'https://utm.calendar.utoronto.ca/section/Anthropology'

In [ ]:
website = 'https://utm.calendar.utoronto.ca/section/Anthropology'
result = requests.get(website)
content = result.text

soup = BeautifulSoup(content, 'lxml')
print(soup.prettify())

In [ ]:
import requests
from bs4 import BeautifulSoup

program_info = {}

# Assume program_list is a list of websites/URLs to scrape
for field in program_list:
    website = field
    result = requests.get(website)
    content = result.text

    soup = BeautifulSoup(content, 'lxml')

    # Extract all <p> tags for additional information if needed
    p_tags = soup.find_all('p')
    about = []
    for p_tag in p_tags:
        about.append(p_tag.get_text(separator="\n"))

    # Find the 'a' tag with id="programs"
    a_tag = soup.find('a', id="programs")
    if a_tag:
        parent_div = a_tag.find_parent('div')

        # Now, within this div, find all the divs with class 'views-row'
        posts = parent_div.find_all('div', class_='views-row')

        # Loop through the posts and extract the desired information
        for p in posts:
            info = []
            # Extract the title (or header) for this section
            title_tag = p.find('h3', class_='js-views-accordion-group-header')

            if title_tag:
                # Get the title text
                title = title_tag.get_text(strip=True)
                print(title)
            else:
                title = "No Title Found"

            # Extract the content within the 'ui-accordion-content' div

            content = p.find('div', class_='view-content')
            if content:
              text_content = content.get_text(separator="\n")
              print(text_content)

            if content:
                # Extract all text, including strong and a tags within the content
                text_content = content.get_text(separator=' ', strip=True)
                info.append(text_content)

            # Only add to the dictionary if title exists and there's info to add
            if title and info:
                program_info[title] = info



In [ ]:
program_info

{}

In [ ]:


for key in program_info.keys():
  print(key)


In [ ]:

with open('program_info.txt', 'w') as f:
  for key, value_list in program_info.items():
    concatenated_string = ' '.join(value_list)
    f.write(f"{key}: {concatenated_string}\n")


In [ ]:


new_course_data = []
for course in course_data:
  if course['Course'] != 'N/A':
    new_course_data.append(course)

course_data = new_course_data


In [ ]:


with open('course_data.txt', 'w') as f:
  for course in course_data:
    if course['Course'] != 'N/A':
      f.write(f"Course: {course['Course']}\n")
      f.write(f"Description: {course['Description']}\n")
      f.write(f"Prerequisites: {course['Prerequisites']}\n\n")


**RAG**

In [ ]:
!pip install langchain langchain_community langchain_chroma

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [ ]:

os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [ ]:
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.1/375.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.0 MB/s eta 0:00:00


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain



In [ ]:
import bs4
from langchain import hub
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [ ]:
# Load the local text document instead of a website
loader = TextLoader("course_data.txt", encoding="utf8")  # Replace with your actual file path
course = loader.load()
loader = TextLoader("program_info (2).txt", encoding="utf8")  # Replace with your actual file path
program = loader.load()

docs = course+ program
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")




In [ ]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:

# Question
rag_chain.invoke("What all the are required courses to complete a comptuer science major during first and second year?")

'First-year required courses for a Computer Science major are CSC108H5, CSC148H5, and MAT102H5. Second-year required courses include CSC207H5, CSC236H5, and one of CSC209H5, CSC258H5, or CSC263H5. Students in the CSC minor are limited to 1.5 credits of computer science courses at the 300/400 level.'

MultiQueryRetriver

In [ ]:
!pip install langchain



In [ ]:
question = "I am interested in a carreer in data science with a what program do you suggest and what courses will i need to take?"

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [ ]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What program would you recommend for pursuing a career in data science, and what specific courses should I consider taking?', '2. Can you suggest a suitable program for someone looking to enter the field of data science, along with a list of essential courses to enroll in?', '3. Which program do you think is best suited for aspiring data scientists, and what courses are essential for building a strong foundation in this field?']


6

Combine

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
# Multi Query: Different Perspectives
template = """You are an AI language model assistant built to act an academic advisor at the university of Toronto. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    store. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. Think about what is important for a student at the university to know, note important course information such as pre requisite and program requirements.
    Provide these alternative questions separated by newlines.
    Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


In [ ]:
from langchain.load import dumps, loads

In [ ]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

question = "I am interested in a carreer in data science with a what sta and mat courses would you suggest taking to build a strong background for an entry level data sceince role?"


retrieval_chain = generate_queries | retriever.map() | get_unique_union
# Tesing a single retriever
# docs = retrieval_chain.invoke({"question":question})
# len(docs)

In [ ]:
from operator import itemgetter

In [ ]:
template = """You are an academic advisor for undegraduate students at UofT, you should provided relevent program and course information
to students. Course recommendations should be alligned with the students academic interests, suggesting courses across different fields of study is encouraged as long as prerequesite courses are stated. provide the same number of course suggestions, minimum four per year, for each first year, second year, third year, and fourth year. Provide an explanation why each course was recommended,
base recomendations of course descriptions. Before providing an aswer ask yourself these questions: Am i suggesting all relevant courses? Am i taking into accounr the students current academic program or future careeer aspirations? Am I suggesting courses from a variety of diciplines not just one area of study?. Take a pause and make sure the answer to these questions is yes. Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})


'For a career in data science, it is important to have a strong background in statistics and mathematics. Here are some recommended STA (Statistics) and MAT (Mathematics) courses that would help you build a solid foundation for an entry-level data science role:\n\nFirst Year:\n1. STA107H5 - An Introduction to Probability and Modelling (No prerequisites)\n2. MAT134H5 - Calculus for Life Sciences I (Prerequisite for higher level math courses)\n3. MAT135H5 - Calculus I (Prerequisite for higher level math courses)\n4. MAT136H5 - Calculus I for Mathematical Sciences (Prerequisite for higher level math courses)\n\nSecond Year:\n1. STA256H5 - Probability and Statistics I (Prerequisite: MAT134H5 or MAT136H5 or equivalent)\n2. STA258H5 - Statistics with Applied Probability (Prerequisite: STA256H5)\n3. MAT232H5 - Calculus of Several Variables (Prerequisite: MAT135H5 or MAT137Y5)\n4. MAT233H5 - Linear Algebra I (Prerequisite: MAT135H5 or MAT137Y5)\n\nThird Year:\n1. STA315H5 - Advanced Statistica

Prompt template

In [ ]:
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant built to act an academic advisor at the university of Toronto. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    store. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. Think about what is important for a student at the university to know, note important course information such as pre requisite and program requirements.
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)
llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
question = "What are the approaches to Task Decomposition?"

In [ ]:

retriever_from_llm = MultiQueryRetriever.from_llm( retriever=vectorstore.as_retriever(), llm=llm, prompt=QUERY_PROMPT )

In [ ]:
unique_docs = retriever_from_llm.invoke(input="I am interested in pursuing a data career in computer science, what are some courses you suggest")
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What courses should I take to prepare for a career in data science within the field of computer science?', '2. Can you recommend specific courses that would be beneficial for someone interested in a data-related career in computer science?', '3. Which courses at the University of Toronto would best align with my goal of pursuing a data career in computer science?', '4. What course options are available at U of T for students looking to specialize in data science within the computer science program?', '5. Could you provide me with a list of courses that would help me build a strong foundation for a data-focused career in computer science at U of T?']


9